In [1]:
import sqlite3
import time 
from minio import Minio
from functools import wraps
import time
import pyarrow.parquet as pq
import pandas as pd
import polars as pl 
import io
import duckdb


BDD = './bdd_actes_budgetaires.db'
client = Minio(
    endpoint='localhost:9000',
    access_key='airflow-local',
    secret_key='airflow-local',
    secure = False
  )


In [2]:
def timeit(func):
    @wraps(func)
    def timeit_wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        total_time = end_time - start_time
        print(f'Function {func.__name__}{args} {kwargs} Took {total_time:.4f} seconds')
        return result
    return timeit_wrapper

# Methode duckdb

Du mal à lire depuis Minio, je passerai du temps dessus plus tard. 

Double join : Un pour quand le code fait référence à Nature, l'autre pour quand le code fait référence à Fonction

In [29]:
@timeit
def methode_duckdb(csv = 'Non') :
 conduck = duckdb.connect(database=':memory:', read_only=False)
 docubudg_t = conduck.read_parquet('./parquet_lignebudget')
 transco_t = conduck.read_parquet('./parquet_pdc')
 requete_duckdb = ''' 
 SELECT
    EXTRACT(YEAR FROM CAST(l.DteStr AS DATE)) AS annee_doc,
    l.Nomenclature,
    l.LibCpte,
    t1.Libelle as Nature_lib,
    t2.Libelle as Fonction_lib,
    MtReal
 FROM 
    docubudg_t as l 
 LEFT JOIN
    transco_t t1 ON EXTRACT(YEAR FROM CAST(l.DteStr AS DATE)) = t1.Annee 
    AND l.Nomenclature = t1.Nomenclature 
    AND l.Nature = t1.Code 
    AND (t1.Categorie = 'Nature' OR t1.Categorie = 'Nature_compte')
 LEFT JOIN
    transco_t t2 ON EXTRACT(YEAR FROM CAST(l.DteStr AS DATE)) = t2.Annee 
    AND l.Nomenclature = t2.Nomenclature 
    AND l.Fonction = t2.Code 
    AND (t2.Categorie = 'Fonction' OR t2.Categorie = 'Fonction_compte')

 '''
 df_transcode_duck = conduck.execute(requete_duckdb).fetchdf()
 conduck.close()
 if csv == 'Oui' : 
  df_transcode_duck.to_csv('Methode_transco_duckdb', index = False) #Cette ligne prend 8sec sans le if 

 return df_transcode_duck


In [23]:
df_transcode_test_duck = methode_duckdb()
#Tourne en général en 0.3sec

Function methode_duckdb() {} Took 0.3682 seconds


In [11]:
df_transcode_test_duck_csv = methode_duckdb(csv = 'Oui')

Function methode_duckdb() {'csv': 'Oui'} Took 1.5418 seconds


# Methode PANDAS

Methode par Merge

In [13]:
@timeit
def methode_pandas_nondict() : 
 docbudg_parquet = io.BytesIO(client.get_object('table-ligne-budget', 'parquet-ligne-budget').read())
 transco_parquet = io.BytesIO(client.get_object('table-plan-de-compte', 'parquet-plan-de-compte').read())
 
 df_l = pd.read_parquet(docbudg_parquet)
 df_t = pd.read_parquet(transco_parquet, columns=['Nomenclature', 'Annee', 'Categorie', 'Code', 'Libelle'])

 #Ca ne devrait pas exister car ça devrait déjà être sous format str dans la bdd, ce n'est pas le cas, "mon mauvais"
 df_l['Fonction'] = df_l['Fonction'].astype(str)
 df_l['Nature'] = df_l['Nature'].astype(str)

 #On coupe le df de transco selon si c'est Nature ou Fonction
 df_t = df_t.copy()
 df_nature = df_t[df_t['Categorie'] == 'Nature']
 df_nature = df_nature.drop_duplicates(subset=['Nomenclature', 'Code'])

 df_fonction = df_t[df_t['Categorie'] == 'Fonction']
 df_fonction = df_fonction.drop_duplicates(subset=['Nomenclature', 'Code'])

 #On commence par merge avec le df transco_nature, 
 # puis on remplace de façon très brute les données de la col Nature (qui est une des clefs de jointure) 
 df_temp_merge_nature = pd.merge(df_l, df_nature, left_on=['Nomenclature', 'Nature'], right_on=['Nomenclature', 'Code'], how='left')
 df_temp_merge_nature['Nature'] = df_temp_merge_nature['Libelle']
 df_temp_merge_nature = df_temp_merge_nature[['IdFichier', 'Nomenclature', 'DteStr', 'Nature', 'LibCpte', 'Fonction', 'MtReal']]

 df_total = pd.merge(df_temp_merge_nature, df_fonction, left_on=['Nomenclature', 'Fonction'], right_on=['Nomenclature', 'Code'], how='left')
 df_total['Fonction'] = df_total['Libelle']
 df_x = df_total[['IdFichier', 'Nomenclature' ,'DteStr', 'LibCpte', 'Nature', 'Fonction', 'MtReal']]

 return df_x

In [14]:
df_pandas_nondict = methode_pandas_nondict()
#Tourne en general entre 0.6 et 0.8sec

Function methode_pandas_nondict() {} Took 0.5689 seconds


#Version dict de données 

In [15]:
@timeit
def methode_pandas_dict() : 
 docbudgpq = io.BytesIO(client.get_object('table-ligne-budget', 'parquet-ligne-budget').read())
 transcopq = io.BytesIO(client.get_object('table-plan-de-compte', 'parquet-plan-de-compte').read())

 df1_l = pd.read_parquet(docbudgpq)
 df1_t = pd.read_parquet(transcopq, columns=['Nomenclature', 'Annee', 'Categorie', 'Code', 'Libelle'])
 
 #df1_l['Fonction'] = df1_l['Fonction'].astype(str)
 #df1_l['Nature'] = df1_l['Nature'].astype(str)

 df1_nature = df1_t[df1_t['Categorie'] == 'Nature']
 dict_nature = df1_nature.set_index('Code')['Libelle'].to_dict()
 df1_l['Nature'] = df1_l['Nature'].map(dict_nature)

 df1_fonction = df1_t[df1_t['Categorie'] == 'Fonction']
 dict_fonction = df1_fonction.set_index('Code')['Libelle'].to_dict()
 df1_l['Fonction'] = df1_l['Fonction'].map(dict_fonction)
 return df1_l

In [17]:
df_ttt = methode_pandas_dict()
df_ttt

Function methode_pandas_dict() {} Took 0.5108 seconds


,IdFichier,Nomenclature,DteStr,Nature,LibCpte,Fonction,MtReal
0,617544,M4-M49_A,2020-07-30,Solde d'exécution de la section d'investisseme...,Solde d'exécution sect° d'investissement,NaN,0.00
1,617544,M4-M49_A,2020-07-30,Solde d'exécution de la section d'investisseme...,Solde d'exécution sect° d'investissement,NaN,8880.38
2,617544,M4-M49_A,2020-07-30,Résultat de fonctionnement reporté,Résultat de fonctionnement reporté,NaN,7583.30
3,617544,M4-M49_A,2020-07-30,Résultat de fonctionnement reporté,Résultat de fonctionnement reporté,NaN,0.00
4,617544,M4-M49_A,2020-07-30,Subventions d'investissement rattachées aux ac...,Subventions d'équipement,NaN,278.00
...,...,...,...,...,...,...,...
287970,634802,M4-M49_D,2020-11-19,Concessions et droits similaires,None,NaN,NaN
287971,634802,M4-M49_D,2020-11-19,Réseaux de voirie,None,NaN,NaN
287972,634802,M4-M49_D,2020-11-19,Avances versées sur commandes d'immobilisation...,None,NaN,NaN
287973,634802,M4-M49_D,2020-11-19,Autres agencements et aménagements,None,NaN,NaN


# Methode Polars 
En cours après une bonne surprise 

In [19]:
#